<a href="https://colab.research.google.com/github/babatundeibukun/DEEP-LEARNING-PROJECT---COMPUTER/blob/main/solving_data_imbalance_with_smote_method_%26_ensemble_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
df  = pd.read_csv('/content/Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.shape

(10000, 14)

In [4]:
df.dtypes

,0
RowNumber,int64
CustomerId,int64
Surname,object
CreditScore,int64
Geography,object
Gender,object
Age,int64
Tenure,int64
Balance,float64
NumOfProducts,int64


In [5]:
#I need to drop the columns that are not needed

df1 = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1 )
df1.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
df1.dtypes

,0
CreditScore,int64
Geography,object
Gender,object
Age,int64
Tenure,int64
Balance,float64
NumOfProducts,int64
HasCrCard,int64
IsActiveMember,int64
EstimatedSalary,float64


In [7]:
df1['Gender'].unique()

array(['Female', 'Male'], dtype=object)

In [8]:
df1['Gender'] = df1['Gender'].replace({'Female' : 0, 'Male' : 1})

<ipython-input-8-5ed6f3f3a3eb>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df1['Gender'] = df1['Gender'].replace({'Female' : 0, 'Male' : 1})


In [9]:
df1 = pd.get_dummies(df1, 'Geography', drop_first = True, dtype= int)
df1.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,0,0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0,1
2,502,0,42,8,159660.80,3,1,0,113931.57,1,0,0
3,699,0,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0,1


#Realized that I need some scaling

In [10]:
from sklearn.preprocessing import MinMaxScaler
sx = MinMaxScaler()


cols_to_scale = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

df1[cols_to_scale] = sx.fit_transform(df[cols_to_scale])

df1[cols_to_scale].head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,EstimatedSalary
0,0.538,0.324324,0.2,0.000000,0.000000,0.506735
1,0.516,0.310811,0.1,0.334031,0.000000,0.562709
2,0.304,0.324324,0.8,0.636357,0.666667,0.569654
3,0.698,0.283784,0.1,0.000000,0.333333,0.469120
4,1.000,0.337838,0.2,0.500246,0.000000,0.395400


In [11]:
df1.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,0.538,0,0.324324,0.2,0.000000,0.000000,1,1,0.506735,1,0,0
1,0.516,0,0.310811,0.1,0.334031,0.000000,0,1,0.562709,0,0,1
2,0.304,0,0.324324,0.8,0.636357,0.666667,1,0,0.569654,1,0,0
3,0.698,0,0.283784,0.1,0.000000,0.333333,0,0,0.469120,0,0,0
4,1.000,0,0.337838,0.2,0.500246,0.000000,1,1,0.395400,0,0,1


#Checking if there is an imbalance in the outcome

In [12]:
df['Exited'].value_counts()

,count
Exited,
0,7963
1,2037


In [13]:
#There is huge imbalance in the dataset, I can solve that with oversampling the minority class, undersampling the majority class, smote or ensemble orr focal loss,
# So i will try all of them one after the other but starting wiht smote since i did undersampling the majority class already .



In [14]:
X = df1.drop('Exited', axis = 1)
X.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain
0,0.538,0,0.324324,0.2,0.000000,0.000000,1,1,0.506735,0,0
1,0.516,0,0.310811,0.1,0.334031,0.000000,0,1,0.562709,0,1
2,0.304,0,0.324324,0.8,0.636357,0.666667,1,0,0.569654,0,0
3,0.698,0,0.283784,0.1,0.000000,0.333333,0,0,0.469120,0,0
4,1.000,0,0.337838,0.2,0.500246,0.000000,1,1,0.395400,0,1


In [15]:
y = df1['Exited']

#SMOTE - METHOD

In [16]:
#first i need to divide the data set into train and test split

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X, y , test_size = 0.2 , random_state = 1234, stratify = y)




In [17]:
X_train.shape

(8000, 11)

In [18]:
X_test.shape

(2000, 11)

In [19]:
y_train.value_counts()

,count
Exited,
0,6370
1,1630


In [20]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(sampling_strategy = 'minority')
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)


In [21]:
y_resampled.value_counts()

,count
Exited,
0,6370
1,6370


In [22]:
X_resampled.shape

(12740, 11)

In [23]:
import tensorflow as tf
from tensorflow import keras

model =  keras.Sequential([
    keras.layers.Dense(12740, input_shape = (11,), activation = 'relu'),
    keras.layers.Dense(100,  activation = 'relu'),
    keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile( optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'] )

model.fit(X_resampled, y_resampled, epochs = 15)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


399/399 ━━━━━━━━━━━━━━━━━━━━ 12s 27ms/step - accuracy: 0.6944 - loss: 0.5826
Epoch 2/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 13s 33ms/step - accuracy: 0.7704 - loss: 0.4706
Epoch 3/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 12s 30ms/step - accuracy: 0.7792 - loss: 0.4551
Epoch 4/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 11s 29ms/step - accuracy: 0.7807 - loss: 0.4499
Epoch 5/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.7919 - loss: 0.4315
Epoch 6/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.7943 - loss: 0.4333
Epoch 7/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - accuracy: 0.8050 - loss: 0.4146
Epoch 8/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 11s 25ms/step - accuracy: 0.8034 - loss: 0.4157
Epoch 9/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.8077 - loss: 0.4042
Epoch 10/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.8127 - loss: 0.4044
Epoch 11/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.8135 - loss: 0.3964
Epoch 12/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 8s

#Now Lets Evaluate

In [24]:
model.evaluate(X_test, y_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7987 - loss: 0.4566


[0.4519938826560974, 0.7944999933242798]

In [25]:
y_preds = np.round(model.predict(X_test))
y_preds


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


array([[1.],
       [1.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [26]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.92      0.82      0.86      1593
           1       0.50      0.71      0.58       407

    accuracy                           0.79      2000
   macro avg       0.71      0.76      0.72      2000
weighted avg       0.83      0.79      0.81      2000



The above result still looks terrible to me though especially for the class_1

#I want to try to use Ensemble

In [29]:
df2 = df1.copy()  #creating a copy of the
df2.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_Germany,Geography_Spain
0,0.538,0,0.324324,0.2,0.000000,0.000000,1,1,0.506735,1,0,0
1,0.516,0,0.310811,0.1,0.334031,0.000000,0,1,0.562709,0,0,1
2,0.304,0,0.324324,0.8,0.636357,0.666667,1,0,0.569654,1,0,0
3,0.698,0,0.283784,0.1,0.000000,0.333333,0,0,0.469120,0,0,0
4,1.000,0,0.337838,0.2,0.500246,0.000000,1,1,0.395400,0,0,1


In [43]:
df2['Exited'].value_counts()

,count
Exited,
0,7963
1,2037


In [31]:
X  = df2.drop('Exited', axis = 1)
X.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain
0,0.538,0,0.324324,0.2,0.000000,0.000000,1,1,0.506735,0,0
1,0.516,0,0.310811,0.1,0.334031,0.000000,0,1,0.562709,0,1
2,0.304,0,0.324324,0.8,0.636357,0.666667,1,0,0.569654,0,0
3,0.698,0,0.283784,0.1,0.000000,0.333333,0,0,0.469120,0,0
4,1.000,0,0.337838,0.2,0.500246,0.000000,1,1,0.395400,0,1


In [32]:
y = df2['Exited']
y.head()

,Exited
0,1
1,0
2,1
3,0
4,0


In [33]:
#the ensemble can only be done on the train data set

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 234, stratify = y)



In [37]:
y_train.value_counts()

,count
Exited,
0,6370
1,1630


In [40]:
df3 = X_train.copy()
df3['Exited'] = y_train

In [41]:
df3.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Exited
6953,0.448,1,0.229730,0.5,0.653081,0.000000,1,1,0.075540,1,0,0
2201,0.500,1,0.229730,0.4,0.572921,0.333333,1,0,0.520374,0,0,0
2686,0.360,0,0.324324,0.0,0.398363,0.000000,0,1,0.486682,1,0,0
6885,0.696,0,0.121622,0.1,0.378324,0.000000,1,1,0.201661,0,0,0
3840,0.778,1,0.540541,0.1,0.440808,0.000000,0,1,0.800632,1,0,1


In [42]:
df3['Exited'].value_counts()


,count
Exited,
0,6370
1,1630


In [45]:
#As seen in our train dataset , there is an huge imbalance in our dataset
#we need to fix that , we shoud , really .

class_0 = df3[df3['Exited'] == 0]
class_0.head()     # classes only consisting of zeros

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Exited
6953,0.448,1,0.229730,0.5,0.653081,0.000000,1,1,0.075540,1,0,0
2201,0.500,1,0.229730,0.4,0.572921,0.333333,1,0,0.520374,0,0,0
2686,0.360,0,0.324324,0.0,0.398363,0.000000,0,1,0.486682,1,0,0
6885,0.696,0,0.121622,0.1,0.378324,0.000000,1,1,0.201661,0,0,0
55,0.538,1,0.337838,0.1,0.499055,0.000000,1,1,0.567049,0,0,0


In [46]:
class_1 = df3[df3['Exited'] == 1]


In [49]:
count_0, count_1 = class_0.shape[0] , class_1.shape[0]
count_0, count_1

(6370, 1630)

I want to use ensemble ,so i will go over the data about 4 times based on 6370/1630

In [50]:
6370/1630

3.9079754601226995

#one way of doing it

In [52]:

df_class_0_under = class_0.sample(count_1)
df_class_0_under.shape


(1630, 12)

#another way of doing it

In [54]:
df_class_0_under = class_0[0:count_1]
df_class_0_under.shape

(1630, 12)

In [55]:
#if the above is possible then we can creat a function for doing this then for the three classes

def get_train_batch(start, end, df_maj_class, df_min_class):
  df_class_0_under = df_maj_class[start:end]
  df_train = pd.concat([df_class_0_under, df_min_class], axis = 0)

  X_train = df_train.drop('Exited', axis = 1)
  y_train = df_train['Exited']


  return X_train, y_train





In [59]:
def ANN(X_train, y_train, X_test, y_test):
  model =  keras.Sequential([
      keras.layers.Dense(12740, input_shape = (11,), activation = 'relu'),
      keras.layers.Dense(100,  activation = 'relu'),
      keras.layers.Dense(1, activation = 'sigmoid')
  ])

  model.compile( optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'] )

  model.fit(X_resampled, y_resampled, epochs = 15)

  print(model.evaluate(X_test, y_test))

  y_preds = model.predict(X_test)
  y_preds = np.round(y_preds)

  print(classification_report(y_test, y_preds))

  return y_preds

#1st train_batch

In [71]:
X_train, y_train = get_train_batch(0, 1630, class_0, class_1 )
y_train.shape

(3260,)

In [72]:
y_pred1 =ANN(X_train, y_train, X_test, y_test)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


399/399 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.6788 - loss: 0.5914
Epoch 2/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - accuracy: 0.7622 - loss: 0.4850
Epoch 3/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.7697 - loss: 0.4666
Epoch 4/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - accuracy: 0.7837 - loss: 0.4496
Epoch 5/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 10s 19ms/step - accuracy: 0.7815 - loss: 0.4418
Epoch 6/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 12s 24ms/step - accuracy: 0.7944 - loss: 0.4392
Epoch 7/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.8013 - loss: 0.4193
Epoch 8/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step - accuracy: 0.8043 - loss: 0.4166
Epoch 9/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.8105 - loss: 0.4057
Epoch 10/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.8092 - loss: 0.3974
Epoch 11/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - accuracy: 0.8159 - loss: 0.3946
Epoch 12/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 8s

#get_train_batch 2nd

In [61]:
X_train, y_train = get_train_batch(1630, 3260, class_0, class_1 )


In [64]:
y_pred2 = ANN(X_train, y_train, X_test, y_test)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


399/399 ━━━━━━━━━━━━━━━━━━━━ 12s 28ms/step - accuracy: 0.6725 - loss: 0.5944
Epoch 2/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 13s 32ms/step - accuracy: 0.7607 - loss: 0.4783
Epoch 3/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 19s 28ms/step - accuracy: 0.7718 - loss: 0.4677
Epoch 4/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 12s 29ms/step - accuracy: 0.7817 - loss: 0.4444
Epoch 5/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 22s 32ms/step - accuracy: 0.7965 - loss: 0.4293
Epoch 6/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 17s 24ms/step - accuracy: 0.7993 - loss: 0.4300
Epoch 7/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - accuracy: 0.7928 - loss: 0.4323
Epoch 8/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.8064 - loss: 0.4120
Epoch 9/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.8062 - loss: 0.4091
Epoch 10/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.8159 - loss: 0.3949
Epoch 11/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.8193 - loss: 0.3993
Epoch 12/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 10s

#get_train_batch 3rd

In [67]:
X_train, y_train = get_train_batch(3260, 4890, class_0, class_1 )


In [68]:
y_pred3 = ANN(X_train, y_train, X_test, y_test)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


399/399 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.6824 - loss: 0.5866
Epoch 2/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 12s 24ms/step - accuracy: 0.7691 - loss: 0.4752
Epoch 3/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.7679 - loss: 0.4661
Epoch 4/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - accuracy: 0.7818 - loss: 0.4511
Epoch 5/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - accuracy: 0.7920 - loss: 0.4360
Epoch 6/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.8023 - loss: 0.4242
Epoch 7/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.8017 - loss: 0.4211
Epoch 8/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.8059 - loss: 0.4183
Epoch 9/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 10s 26ms/step - accuracy: 0.8071 - loss: 0.4080
Epoch 10/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - accuracy: 0.8251 - loss: 0.3888
Epoch 11/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - accuracy: 0.8189 - loss: 0.3908
Epoch 12/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 10s 2

#get train_batch 4th

In [69]:
X_train, y_train = get_train_batch(4890, 6370, class_0, class_1)


In [70]:
y_pred4 = ANN(X_train, y_train, X_test, y_test)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


399/399 ━━━━━━━━━━━━━━━━━━━━ 13s 29ms/step - accuracy: 0.6864 - loss: 0.5828
Epoch 2/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 10s 26ms/step - accuracy: 0.7610 - loss: 0.4763
Epoch 3/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step - accuracy: 0.7749 - loss: 0.4613
Epoch 4/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 11s 23ms/step - accuracy: 0.7781 - loss: 0.4576
Epoch 5/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 11s 26ms/step - accuracy: 0.7906 - loss: 0.4381
Epoch 6/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 18s 20ms/step - accuracy: 0.7967 - loss: 0.4305
Epoch 7/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - accuracy: 0.8010 - loss: 0.4208
Epoch 8/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.7999 - loss: 0.4200
Epoch 9/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.8128 - loss: 0.4144
Epoch 10/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 8s 19ms/step - accuracy: 0.8075 - loss: 0.4059
Epoch 11/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 10s 20ms/step - accuracy: 0.8133 - loss: 0.4016
Epoch 12/15
399/399 ━━━━━━━━━━━━━━━━━━━━ 11

#Now choosing the best scores

In [81]:
y_preds_final = y_pred1.copy()

for i in range(len(y_pred1)):
  n_ones = y_pred1[i] + y_pred2[i] + y_pred3[i] + y_pred4[i]
  if n_ones >= 2 :
    y_preds_final[i] =   1
  else :
    y_preds_final[i] =   0



In [82]:
print(classification_report(y_test, y_preds_final))

              precision    recall  f1-score   support

           0       0.94      0.84      0.89      1593
           1       0.56      0.80      0.66       407

    accuracy                           0.83      2000
   macro avg       0.75      0.82      0.77      2000
weighted avg       0.86      0.83      0.84      2000

